In [363]:
import pandas as pd

Pandas é uma biblioteca para trabalhar com leitura e manipulação de dados. Utilizamos para leitura do arquivo de dados disponibilizado.


In [364]:
dataframe = pd.read_excel("dataset.xlsx")

In [365]:
dataframe

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Respiratory Syncytial Virus,Influenza A,Influenza B,Parainfluenza 1,CoronavirusNL63,Rhinovirus/Enterovirus,Mycoplasma pneumoniae,Coronavirus HKU1,Parainfluenza 3,Chlamydophila pneumoniae,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,Parainfluenza 2,Neutrophils,...,Urine - Esterase,Urine - Aspect,Urine - pH,Urine - Hemoglobin,Urine - Bile pigments,Urine - Ketone Bodies,Urine - Nitrite,Urine - Density,Urine - Urobilinogen,Urine - Protein,Urine - Sugar,Urine - Leukocytes,Urine - Crystals,Urine - Red blood cells,Urine - Hyaline cylinders,Urine - Granular cylinders,Urine - Yeasts,Urine - Color,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,0.102004,0.318366,-0.950790,-0.094610,-0.223767,-0.292269,1.482158,0.166192,0.357547,-0.625073,-0.140648,not_detected,not_detected,not_detected,not_detected,not_detected,detected,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,-0.619086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,detected,NaN,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,not_detected,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

Devido a presença de campos com o valor NaN, vamos substituir esses valores por 0 para realizar o treinamento das árvores.

In [366]:
dataframe = dataframe.fillna(0)

# Exploração de Dados
A função describe() do pandas foi utilizada para mostrar uma visão geral a respeito dos dados estatísticos presentes na tabela. 

A função info() apresenta dados sobre a tabela em si (tipos de campos presentes, quantidade de colunas, etc).



In [367]:
dataframe.describe()

,Patient age quantile,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Mycoplasma pneumoniae,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Density,Urine - Sugar,Urine - Red blood cells,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
count,5644.000000,5644.000000,5644.000000,5644.000000,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5644.0,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5644.0,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03,5.644000e+03
mean,9.318391,0.013997,0.008859,0.007264,-2.335732e-10,-1.710835e-09,-3.770503e-11,7.894130e-10,8.985679e-10,-8.390814e-10,1.082473e-10,6.629928e-10,-7.075675e-10,-3.683047e-10,7.686216e-10,-4.432198e-10,-3.428930e-10,1.088413e-09,2.605525e-10,0.0,5.370286e-10,-4.695390e-10,2.492080e-10,-5.017987e-10,1.339890e-09,7.366095e-10,1.084123e-10,-2.178146e-11,-4.009769e-11,-8.976603e-11,3.887661e-10,9.768656e-11,-5.197849e-11,1.689714e-10,-3.696248e-11,-4.663213e-10,3.636844e-10,-2.590674e-11,-1.905878e-10,0.0,-2.197948e-10,8.658957e-11,-1.064322e-10,4.488302e-11,-4.603809e-10,1.267285e-10,1.478499e-10,2.376160e-10,0.0,-3.168213e-11,0.0,9.372630e-11,0.0,-1.584106e-11,-1.115475e-10,3.102208e-11,0.0,-1.056071e-11,-1.194680e-10,2.970200e-11,-7.920532e-12,-5.280355e-12,0.0,-1.320089e-11,-6.600441e-13,4.026270e-11,-7.920532e-12,1.980133e-12,-3.574553e-11,2.904195e-11,-1.181479e-10,1.650111e-11,2.244151e-11,2.508169e-11
std,5.777903,0.117489,0.093713,0.084929,3.268915e-01,3.268915e-01,3.266203e-01,3.258055e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.266203e-01,3.263489e-01,3.266203e-01,1.919891e-01,0.0,3.015113e-01,2.652407e-01,2.994472e-01,2.741119e-01,2.564082e-01,2.560624e-01,1.996808e-01,2.001240e-01,1.646610e-01,1.795894e-01,1.795894e-01,1.795894e-01,1.597446e-01,9.413042e-02,8.419280e-02,1.552439e-01

In [368]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Columns: 111 entries, Patient ID to ctO2 (arterial blood gas analysis)
dtypes: float64(70), int64(4), object(37)
memory usage: 4.8+ MB


Acessamos a coluna que contém o diagnóstico final e atribuimos à variável diagnostic.

In [369]:
diagnostic = dataframe[["SARS-Cov-2 exam result"]]

In [370]:
diagnostic

,SARS-Cov-2 exam result
0,negative
1,negative
2,negative
3,negative
4,negative
...,...
5639,positive
5640,negative
5641,negative
5642,negative


# Pré processamento dos dados

Removemos todas as colunas que não são do tipo numérico visto que a função de treino da floresta randômica não sabe lidar com dados do tipo *string*.

In [371]:
cols_to_remove = []

for col in dataframe.columns:
    try:
        _ = dataframe[col].astype(float)
    except ValueError:
        cols_to_remove.append(col)
        pass

# keep only the columns in dataframe that do not contain string
dataframe = dataframe[[col for col in dataframe.columns if col not in cols_to_remove]]

Verificando se todos as colunas restantes possuem valores numéricos e não nulos.

In [372]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Data columns (total 74 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Patient age quantile                                   5644 non-null   int64  
 1   Patient addmited to regular ward (1=yes, 0=no)         5644 non-null   int64  
 2   Patient addmited to semi-intensive unit (1=yes, 0=no)  5644 non-null   int64  
 3   Patient addmited to intensive care unit (1=yes, 0=no)  5644 non-null   int64  
 4   Hematocrit                                             5644 non-null   float64
 5   Hemoglobin                                             5644 non-null   float64
 6   Platelets                                              5644 non-null   float64
 7   Mean platelet volume                                   5644 non-null   float64
 8   Red blood Cells                                 

Atribuimos à variável predictors os dados de laboratório, ou seja, os dados das colunas restantes com exceção da coluna de diagnóstico e das colunas removidas no pré processamento por conterem valores não numéricos.

In [373]:
predictors = dataframe.drop(columns=['Patient addmited to regular ward (1=yes, 0=no)', 'Patient addmited to semi-intensive unit (1=yes, 0=no)','Patient addmited to intensive care unit (1=yes, 0=no)'])

In [374]:
predictors

,Patient age quantile,Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Mycoplasma pneumoniae,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Density,Urine - Sugar,Urine - Red blood cells,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,0.236515,-0.022340,-0.517413,0.010677,0.102004,0.318366,-0.950790,-0.094610,-0.223767,-0.292269,1.482158,0.166192,0.357547,-0.625073,-0.140648,0.0,-0.619086,1.198059,-0.147895,2.089928,-0.305787,0.862512,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

# Divisão do conjunto de dados em Teste e Treino
Utilizamos a biblioteca train_test_split do sklearn para dividir os dados em 30% teste e 70% treino.

In [375]:
from sklearn.model_selection import train_test_split

In [376]:
predictors_train, predictors_test, diagnostic_train, diagnostic_test = train_test_split(predictors, diagnostic, test_size=0.30, random_state=88)

A função train_test_split() com o 

*   Item da lista
*   Item da lista

parâmetro test_size=0.30 retorna 4 valores que são 4 dataframes da respectiva ordem: 

*  70% dos dados para construir as árvores (preditores),
*  30% dos dados para teste (predictors_test),
*  os diagnósticos referentes aos 70% dos dados preditores (diagnostic_train) 
* e os diagnósticos referentes aos 30% dos dados de teste (diagnostic_test).

# Declaração da *Random Forest* e treinamento dela com os dados de treino

In [377]:
from sklearn.ensemble import RandomForestClassifier 

In [378]:
rf_model = RandomForestClassifier(n_estimators=100)

In [379]:
rf_model.fit(predictors_train,diagnostic_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Verificando a relevância das colunas dos preditores

Para verificar as 10 colunas (testes/variáveis) mais relevantes para o diagnóstico final, criamos um dataframe com duas colunas (o nome da feature e a respectiva importância no modelo). Depois, ordenamos em ordem da maior para a menor.

In [380]:
columns_relevance = pd.DataFrame({'feature': list(predictors.columns),
                   'importance': rf_model.feature_importances_}).\
                    sort_values('importance', ascending = False)


In [381]:
columns_relevance.head(n=10)

,feature,importance
0,Patient age quantile,0.191283
8,Leukocytes,0.071378
13,Monocytes,0.068363
3,Platelets,0.057373
19,Proteina C reativa mg/dL,0.039146
11,Eosinophils,0.032021
1,Hematocrit,0.025452
10,Mean corpuscular hemoglobin (MCH),0.025262
5,Red blood Cells,0.025170
2,Hemoglobin,0.024588


# Verificando a precisão da predição do diagnóstico dos pacientes

Importamos a função classification_report() da biblioteca sklearn para visualizar as diversas métricas de acurácia obtidas. 

In [382]:
from sklearn.metrics import classification_report

Usamos o método predict() para testar no nosso modelo criado e treinado (rf_model) o conjunto de 30% dos dados originais que foram separados para teste anteriormente.

In [383]:
rf_predict = rf_model.predict(predictors_test)

O método classification_report() compara o rótulo objetivo (diagnóstico) do conjunto de 30% do original com as previsões do modelo. Por fim, imprimimos o resultado desse método.

In [384]:
print(classification_report(diagnostic_test, rf_predict))

              precision    recall  f1-score   support

    negative       0.91      1.00      0.95      1529
    positive       1.00      0.03      0.06       165

    accuracy                           0.91      1694
   macro avg       0.95      0.52      0.50      1694
weighted avg       0.91      0.91      0.86      1694



# Análise dos resultados obtidos

Pode-se observar que o nosso modelo consegue obter com 91% de precisão, os casos em que o diagnóstico é negativo para COVID-19. Além disso, obteve 100% de precisão para prever os casos positivos de COVID-19.




Entretanto, a métrica *recall* para o diagnóstico positivo está baixa, o que indica que os dados utilizados para o treinamento do modelo estão desbalanceados (existem muito mais diagnósticos negativos do que positivos no *dataframe* utilizado). Isso pode ser verificado ao realizar a contagem dos resultados dos diagnósticos (PCR).

In [385]:
diagnostic.value_counts()

SARS-Cov-2 exam result
negative                  5086
positive                   558
dtype: int64

# Criação de nova coluna
A tabela dada pelo professor possui tres colunas boleanas para sinalizar se o paciente foi ou não internado em alguma das três alas do hospital. Porem, não apresenta dados para caso o paciente esteja fazendo acompanhamento em casa. Para isso, iremos adicionar uma coluna com valores de 0 a 3 para indicar se o paciente foi ou não internado, onde 


*   0 = acompanhados em casa
*   1 = internados na enfermaria
*   2 = internados em unidade semi-intensiva
e
*   3 = internados em unidade intensiva

Iremos deduzir que os pacientes que não foram internados nem na enfermaria, nem na unidade semi-intensiva e nem na unidade intensiva serão acompanhados em casa.



In [386]:
def unit_selector(row):
  if row['Patient addmited to regular ward (1=yes, 0=no)'] == 1:
    return 1
  elif row['Patient addmited to semi-intensive unit (1=yes, 0=no)'] ==1:
    return 2
  elif row['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1:
    return 3
  else :
    return 0

In [387]:
dataframe['addimited unit'] = dataframe.apply(lambda row:unit_selector(row), axis=1)

In [388]:
dataframe

,Patient age quantile,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Mycoplasma pneumoniae,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Density,Urine - Sugar,Urine - Red blood cells,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis),addimited unit
0,13,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,17,0,0,0,0.236515,-0.022340,-0.517413,0.010677,0.102004,0.318366,-0.950790,-0.094610,-0.223767,-0.292269,1.482158,0.166192,0.357547,-0.625073,-0.140648,0.0,-0.619086,1.198059,-0.147895,2.089928,-0.305787,0.862512,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,8,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,5,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,15,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,.

# Previsão de ala

Analogamente, utilizamos como diagnóstico a coluna "addimited unit" para predizer em qual ala o paciente foi acompanhado: 1) internados em enfermaria, 2) internados em unidade semi-intensiva, 3) internados em unidade intensiva, 4) acompanhados em casa .

In [389]:
diagnostic_ward = dataframe[["addimited unit"]]

In [390]:
diagnostic_ward

,addimited unit
0,0
1,0
2,0
3,0
4,0
...,...
5639,0
5640,0
5641,0
5642,0


In [391]:
diagnostic_ward.value_counts()

addimited unit
0                 5474
1                   79
2                   50
3                   41
dtype: int64

O número de casos em que a pessoa foi acompanhada em casa é significativamente maior do que os outros.

In [392]:
dataframe = dataframe.drop(columns=['Patient addmited to regular ward (1=yes, 0=no)', 'Patient addmited to semi-intensive unit (1=yes, 0=no)','Patient addmited to intensive care unit (1=yes, 0=no)', 'addimited unit'])

In [393]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5644 entries, 0 to 5643
Data columns (total 71 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Patient age quantile                              5644 non-null   int64  
 1   Hematocrit                                        5644 non-null   float64
 2   Hemoglobin                                        5644 non-null   float64
 3   Platelets                                         5644 non-null   float64
 4   Mean platelet volume                              5644 non-null   float64
 5   Red blood Cells                                   5644 non-null   float64
 6   Lymphocytes                                       5644 non-null   float64
 7   Mean corpuscular hemoglobin concentration (MCHC)  5644 non-null   float64
 8   Leukocytes                                        5644 non-null   float64
 9   Basophils          

In [394]:
predictors_ward = dataframe

In [395]:
predictors

,Patient age quantile,Hematocrit,Hemoglobin,Platelets,Mean platelet volume,Red blood Cells,Lymphocytes,Mean corpuscular hemoglobin concentration (MCHC),Leukocytes,Basophils,Mean corpuscular hemoglobin (MCH),Eosinophils,Mean corpuscular volume (MCV),Monocytes,Red blood cell distribution width (RDW),Serum Glucose,Mycoplasma pneumoniae,Neutrophils,Urea,Proteina C reativa mg/dL,Creatinine,Potassium,Sodium,Alanine transaminase,Aspartate transaminase,Gamma-glutamyltransferase,Total Bilirubin,Direct Bilirubin,Indirect Bilirubin,Alkaline phosphatase,Ionized calcium,Magnesium,pCO2 (venous blood gas analysis),Hb saturation (venous blood gas analysis),Base excess (venous blood gas analysis),pO2 (venous blood gas analysis),Fio2 (venous blood gas analysis),Total CO2 (venous blood gas analysis),pH (venous blood gas analysis),HCO3 (venous blood gas analysis),Rods #,Segmented,Promyelocytes,Metamyelocytes,Myelocytes,Myeloblasts,Urine - Density,Urine - Sugar,Urine - Red blood cells,Partial thromboplastin time (PTT),Relationship (Patient/Normal),International normalized ratio (INR),Lactic Dehydrogenase,"Prothrombin time (PT), Activity",Vitamin B12,Creatine phosphokinase (CPK),Ferritin,Arterial Lactic Acid,Lipase dosage,D-Dimer,Albumin,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17,0.236515,-0.022340,-0.517413,0.010677,0.102004,0.318366,-0.950790,-0.094610,-0.223767,-0.292269,1.482158,0.166192,0.357547,-0.625073,-0.140648,0.0,-0.619086,1.198059,-0.147895,2.089928,-0.305787,0.862512,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [396]:
predictors_ward_train, predictors_ward_test, diagnostic_ward_train, diagnostic_ward_test = train_test_split(predictors_ward, diagnostic_ward, test_size=0.30, random_state=88)

In [397]:
rf_ward_model = RandomForestClassifier(n_estimators=100)

In [398]:
rf_ward_model.fit(predictors_ward_train,diagnostic_ward_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [399]:
columns_relevance_ward = pd.DataFrame({'feature': list(predictors_ward.columns),
                   'importance': rf_ward_model.feature_importances_}).\
                    sort_values('importance', ascending = False)

In [400]:
columns_relevance_ward.head()

,feature,importance
19,Proteina C reativa mg/dL,0.078876
3,Platelets,0.040879
11,Eosinophils,0.037662
0,Patient age quantile,0.036031
35,pO2 (venous blood gas analysis),0.035912


In [401]:
rf_predict_ward = rf_ward_model.predict(predictors_ward_test)

In [402]:
print(classification_report(diagnostic_ward_test, rf_predict_ward))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1643
           1       0.00      0.00      0.00        23
           2       0.12      0.06      0.08        16
           3       1.00      0.17      0.29        12

    accuracy                           0.97      1694
   macro avg       0.53      0.31      0.34      1694
weighted avg       0.96      0.97      0.96      1694



Podemos observar que podemos predizer com **98% de precisão** os casos em que o paciente tem que ser **acompanhado em casa**. Para os demais casos, obtemos as seguintes precisões: 

1. 0% de precisão para pacientes que devem ser internados em enfermaria 
2. 12% de precisão para pacientes que devem ser internados em unidade semi-intensiva
3. 100% de precisão para pacientes que devem ser internados em unidade intensiva

Entretanto, a métrica *recall* para a previsão do tipo de acompanhamento indicado está baixa para o casos 1, 2 e 3. Isso indica que os dados estão desbalanceados, visto que a Floresta Randômica (*Random Forest*) foi treinada com muito mais exemplos em que o paciente teria de ser acompanhado em casa.

Dessa forma, o único caso que podemos prever de forma segura é o caso em que o paciente tem que ser acompanhado em casa. Os demais apresentam níveis de acurácia muito abaixo do ideal. Considerando o contexto médico em que o algoritmo é utilizado, o ideal é que a acurácia fosse mais confiável para diagnosticar corretamente como o paciente deve ser acompanhado. 

# Referências

*   Removendo strings de um dataframe: https://stackoverflow.com/questions/42335385/delete-every-column-that-contains-a-string-in-dataframe
*   Modelo do RandomForest do Scikit-learn: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
*   Leitura de arquivos Xlsx com pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
*   Verificando relevância das colunas: https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76